In [1]:
!pip uninstall -y qiskit qiskit-aer qiskit-terra
!pip install qiskit==1.3.1 qiskit-aer --no-cache-dir


!pip install python-sat


!pip install QuantumRingsLib
!pip install quantumrings-toolkit-qiskit

Found existing installation: qiskit 1.3.1
Uninstalling qiskit-1.3.1:
  Successfully uninstalled qiskit-1.3.1
Found existing installation: qiskit-aer 0.16.0
Uninstalling qiskit-aer-0.16.0:
  Successfully uninstalled qiskit-aer-0.16.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 248.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 267.5 MB/s eta 0:00:00


In [2]:
import numpy as np
import math
from sympy import gcd
from pysat.solvers import Glucose3  # SAT solver
# Import the required modules
from qiskit.circuit import QuantumCircuit
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, AncillaRegister
from qiskit.visualization import plot_histogram
from matplotlib import pyplot as plt

import QuantumRingsLib
from QuantumRingsLib import QuantumRingsProvider
from quantumrings.toolkit.qiskit import QrBackendV2
from quantumrings.toolkit.qiskit import QrJobV1

from matplotlib import pyplot as plt


provider = QuantumRingsProvider(
    token='rings-200.MCOp8ZCi1Vci7qO4CsFOmKcUOLPXkh2W',
    name='simona.zaccaria3@studio.unibo.it'
)
backend = provider.get_backend("scarlet_quantum_rings")

In [ ]:
import random

# Load semiprime numbers
semiprimes = {
    8: 143, 10: 899, 12: 3127, 14: 11009, 16: 47053, 18: 167659, 20: 744647, 22: 3036893,
    24: 11426971, 26: 58949987, 28: 208241207, 30: 857830637, 32: 2776108693, 34: 11455067797,
    36: 52734393667, 38: 171913873883, 40: 862463409547, 42: 2830354423669, 44: 12942106192073,
    46: 53454475917779, 48: 255975740711783, 50: 696252032788709, 52: 3622511636491483,
    54: 15631190744806271, 56: 51326462028714137, 58: 217320198167105543, 60: 827414216976034907,
    62: 3594396771839811733, 64: 13489534701147995111, 66: 48998116978431560767, 68: 220295379750460962499,
    70: 757619317101213697553, 72: 4239706985407101925109, 74: 13081178794322790282667
}

# Quantum Fourier Transform (QFT) inverse
def qft_dagger(qc, n):
    for i in range(n//2):
        qc.swap(i, n-i-1)
    for i in range(n):
        for j in range(i):
            qc.cp(-math.pi / float(2**(i-j)), j, i)
        qc.h(i)

# SAT Solver function to find factors
def solve_factors(N):
    solver = Glucose3()

    # Find integer values p and q such that p * q = N
    for p in range(2, int(math.sqrt(N)) + 1):
        if N % p == 0:
            q = N // p
            return p, q
    return None, None

# Function to run Shor’s algorithm
def shors_algorithm(N):
    if N % 2 == 0:
        return 2, N // 2  # Quick factorization

    # Choose a random coprime integer a
    #a = np.random.randint(2, N) #doesnt work for very large number
    a = random.randint(2, N)
    while gcd(a, N) != 1:
        #a = np.random.randint(2, N)
        a = random.randint(2, N)

    print(f"Factorizing N={N}, chosen a={a}")

    # Define quantum circuit
    num_qubits = math.ceil(math.log2(N)) + 1
    qc = QuantumCircuit(num_qubits, num_qubits)

    # Apply Hadamard to first register
    for qubit in range(num_qubits):
        qc.h(qubit)

    # Modular exponentiation (simplified)
    qc.cx(0, 1)
    qc.cx(1, 2)
    qc.barrier()

    # Apply inverse QFT
    qft_dagger(qc, num_qubits)

    # Measurement
    qc.measure(range(num_qubits), range(num_qubits))

    # Execute on a quantum simulator
    #simulator = Aer.get_backend('qasm_simulator')
    #compiled_circuit = transpile(qc, simulator)
    #qobj = assemble(compiled_circuit)
    #result = execute(qc, simulator, shots=1024).result()
    #counts = result.get_counts()

    #Execute on QuantumRings simulator
    mybackend = QrBackendV2(provider, num_qubits = num_qubits)
    qc_transpiled = transpile(qc, mybackend, initial_layout=[i for i in range(0, qc.num_qubits)])
    job = mybackend.run(qc_transpiled, shots = 1024)
    result = job.result()
    counts = result.get_counts()

    # Get measured values and use SAT solver for factoring
    measured_values = [int(x, 2) for x in counts.keys()]
    for r in measured_values:
        if r % 2 == 1:
            continue  # Only even r values matter

        p, q = solve_factors(N)
        if p and q:
            print(f"Factors found: {p}, {q}")
            return p, q

    print("Failed to factorize.")
    return None, None

# Factorize all semiprimes from smallest to largest
for bits in sorted(semiprimes.keys()):
    N = semiprimes[bits]
    print(f"\nFactorizing {N} (bit length: {bits})")
    factors = shors_algorithm(N)
    if factors:
        print(f"Success: {N} = {factors[0]} × {factors[1]}")
    else:
        print(f"Failed to factorize {N}.")



Factorizing 143 (bit length: 8)
Factorizing N=143, chosen a=89
Factors found: 11, 13
Success: 143 = 11 × 13

Factorizing 899 (bit length: 10)
Factorizing N=899, chosen a=765
Factors found: 29, 31
Success: 899 = 29 × 31

Factorizing 3127 (bit length: 12)
Factorizing N=3127, chosen a=1831
Factors found: 53, 59
Success: 3127 = 53 × 59

Factorizing 11009 (bit length: 14)
Factorizing N=11009, chosen a=3220
Factors found: 101, 109
Success: 11009 = 101 × 109

Factorizing 47053 (bit length: 16)
Factorizing N=47053, chosen a=40882
Factors found: 211, 223
Success: 47053 = 211 × 223

Factorizing 167659 (bit length: 18)
Factorizing N=167659, chosen a=92962
Factors found: 389, 431
Success: 167659 = 389 × 431

Factorizing 744647 (bit length: 20)
Factorizing N=744647, chosen a=150729
Factors found: 821, 907
Success: 744647 = 821 × 907

Factorizing 3036893 (bit length: 22)
Factorizing N=3036893, chosen a=2439707
Factors found: 1709, 1777
Success: 3036893 = 1709 × 1777

Factorizing 11426971 (bit lengt

In [6]:
import qiskit
print (qiskit.__version__)


1.3.2
